# Análise de Candidatos ao Vestibular da UEA

-  Análise exploratória
    - [X] Verificar existência de dados nulos
    - [X] Formatar ``datanasc`` de str para datetime
    - [X] Formatar ``classificacao`` de float para int
    - [X] Criar atributo ``idade``
    - [X] Formatar atributo ``id_referencia_grupo``
    - [ ] Ordenar (na ordem que aparece na leitura do PDF) dataframe considerando: 
        1. Por ``ano``
        2. Por ``curso_nome_referencia``
        3. Por ``id_curso_referencia``
        4. Por ``classificacao`` 


## Pré-processamento dos dados

In [42]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import euclidean

from datetime import datetime, date

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

### Carregando o dataset

In [43]:
data = pd.read_csv('../data/csv/candidates_info.csv', sep=',', encoding='utf-8')

In [44]:
data.head(5)

,numinscricao,nome,datanasc,classificacao,opcao_curso_2,opcao_3_curso,habilidades,nota_final,nota_cg,nota_etapa_1,disc1,disc2,disc3,nota_ce,nota_redacao,nota_etapa_2,opcao_curso_1,id_referencia_curso,nome_referencia_curso,ano
0,6354130-0,EDIJANE PINHEIRO MARTINS,04/03/92,313.0,No,No,No,"31,161","19,048","19,048",3,5,6,"28,00","15,27","43,273",4069,4069,Odontologia - Integral - Manaus,2019
1,6318761-2,JOEL DOS SANTOS DE SOUZA,09/03/97,314.0,No,No,No,"31,154","29,762","29,762",3,1,4,"16,00","16,55","32,545",4069,4069,Odontologia - Integral - Manaus,2019
2,6379123-4,CARINA DAS CHAGAS SEIXAS,23/10/96,315.0,No,No,No,"31,154","29,762","29,762",3,3,2,"16,00","16,55","32,545",4069,4069,Odontologia - Integral - Manaus,2019
3,6549853-4,IRLA DA SILVA PEREIRA,07/02/01,316.0,No,No,No,"31,113","30,952","30,952",4,0,4,"16,00","15,27","31,273",4069,4069,Odontologia - Integral - Manaus,2019
4,6393350-0,IVANIR LIMA PEREIRA,11/03/91,317.0,No,No,No,"31,113","30,952","30,952",2,3,3,"16,00","15,27","31,273",4069,4069,Odontologia - Integral - Manaus,2019


## Análise exploratória dos dados

In [45]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 184389 entries, 0 to 184388
Data columns (total 20 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   numinscricao           184389 non-null  object 
 1   nome                   184389 non-null  object 
 2   datanasc               184389 non-null  object 
 3   classificacao          184389 non-null  float64
 4   opcao_curso_2          184389 non-null  object 
 5   opcao_3_curso          184389 non-null  object 
 6   habilidades            184389 non-null  object 
 7   nota_final             184389 non-null  object 
 8   nota_cg                184389 non-null  object 
 9   nota_etapa_1           184389 non-null  object 
 10  disc1                  184389 non-null  int64  
 11  disc2                  184389 non-null  int64  
 12  disc3                  184389 non-null  int64  
 13  nota_ce                184389 non-null  object 
 14  nota_redacao           184389 non-nu

#### Verificando a existência de colunas com dados nulos

In [46]:
data.isnull().sum()

numinscricao             0
nome                     0
datanasc                 0
classificacao            0
opcao_curso_2            0
opcao_3_curso            0
habilidades              0
nota_final               0
nota_cg                  0
nota_etapa_1             0
disc1                    0
disc2                    0
disc3                    0
nota_ce                  0
nota_redacao             0
nota_etapa_2             0
opcao_curso_1            0
id_referencia_curso      0
nome_referencia_curso    0
ano                      0
dtype: int64

#### Formatando coluna 'datanasc' para o formato date

In [47]:
data['datanasc'] = pd.to_datetime(data['datanasc'], format='%d/%m/%y')
data.head(4)

,numinscricao,nome,datanasc,classificacao,opcao_curso_2,opcao_3_curso,habilidades,nota_final,nota_cg,nota_etapa_1,disc1,disc2,disc3,nota_ce,nota_redacao,nota_etapa_2,opcao_curso_1,id_referencia_curso,nome_referencia_curso,ano
0,6354130-0,EDIJANE PINHEIRO MARTINS,1992-03-04,313.0,No,No,No,"31,161","19,048","19,048",3,5,6,"28,00","15,27","43,273",4069,4069,Odontologia - Integral - Manaus,2019
1,6318761-2,JOEL DOS SANTOS DE SOUZA,1997-03-09,314.0,No,No,No,"31,154","29,762","29,762",3,1,4,"16,00","16,55","32,545",4069,4069,Odontologia - Integral - Manaus,2019
2,6379123-4,CARINA DAS CHAGAS SEIXAS,1996-10-23,315.0,No,No,No,"31,154","29,762","29,762",3,3,2,"16,00","16,55","32,545",4069,4069,Odontologia - Integral - Manaus,2019
3,6549853-4,IRLA DA SILVA PEREIRA,2001-02-07,316.0,No,No,No,"31,113","30,952","30,952",4,0,4,"16,00","15,27","31,273",4069,4069,Odontologia - Integral - Manaus,2019


In [48]:
data['classificacao'] = data['classificacao'].astype(int)

In [49]:
data.sample(2)

,numinscricao,nome,datanasc,classificacao,opcao_curso_2,opcao_3_curso,habilidades,nota_final,nota_cg,nota_etapa_1,disc1,disc2,disc3,nota_ce,nota_redacao,nota_etapa_2,opcao_curso_1,id_referencia_curso,nome_referencia_curso,ano
78640,8297693-7,VIVIANE TOMAZ MONTEIRO,1983-08-31,50,02058,03058,No,"36,046","25,000","25,000",4,3,7,"28,00","19,09","47,091",1058,2058,Licenciatura em Pedagogia - Noturno - Manaus,2020
72320,8319007-4,JOANA PIMENTEL FARIAS,1998-04-20,2,02060,03060,No,"43,427","29,762","29,762",4,5,10,"38,00","19,09","57,091",1060,1060,Licenciatura em Química - Matutino - Parintins,2020


## Criando novos atributos

In [50]:
def calculate_age(row) -> int:
    born = row['datanasc']
    year = row['ano']
    age = year - born.year
    return age

In [51]:
data['idade'] = data.apply(calculate_age, axis=1)
data.drop(columns=['datanasc'], axis=1, inplace=True)
data.head(5)

,numinscricao,nome,classificacao,opcao_curso_2,opcao_3_curso,habilidades,nota_final,nota_cg,nota_etapa_1,disc1,disc2,disc3,nota_ce,nota_redacao,nota_etapa_2,opcao_curso_1,id_referencia_curso,nome_referencia_curso,ano,idade
0,6354130-0,EDIJANE PINHEIRO MARTINS,313,No,No,No,"31,161","19,048","19,048",3,5,6,"28,00","15,27","43,273",4069,4069,Odontologia - Integral - Manaus,2019,27
1,6318761-2,JOEL DOS SANTOS DE SOUZA,314,No,No,No,"31,154","29,762","29,762",3,1,4,"16,00","16,55","32,545",4069,4069,Odontologia - Integral - Manaus,2019,22
2,6379123-4,CARINA DAS CHAGAS SEIXAS,315,No,No,No,"31,154","29,762","29,762",3,3,2,"16,00","16,55","32,545",4069,4069,Odontologia - Integral - Manaus,2019,23
3,6549853-4,IRLA DA SILVA PEREIRA,316,No,No,No,"31,113","30,952","30,952",4,0,4,"16,00","15,27","31,273",4069,4069,Odontologia - Integral - Manaus,2019,18
4,6393350-0,IVANIR LIMA PEREIRA,317,No,No,No,"31,113","30,952","30,952",2,3,3,"16,00","15,27","31,273",4069,4069,Odontologia - Integral - Manaus,2019,28


In [52]:
data.sample(8)

,numinscricao,nome,classificacao,opcao_curso_2,opcao_3_curso,habilidades,nota_final,nota_cg,nota_etapa_1,disc1,disc2,disc3,nota_ce,nota_redacao,nota_etapa_2,opcao_curso_1,id_referencia_curso,nome_referencia_curso,ano,idade
149861,2338680-0,CARLA GEOVANNA PEREIRA FRANCA,751,02015,03015,No,"32,492","22,619","22,619",6,3,2,"22,00","20,36","42,364",1015,2015,Direito - Vespertino - Manaus,2022,18
50153,6768460-2,MATHEUS DE MIRANDA SIQUEIRA,91,02026,03026,No,"38,303","33,333","33,333",5,5,4,"28,00","15,27","43,273",1026,1026,Engenharia Química - Integral - Manaus,2019,17
88944,8578253-0,PEDRO VICTOR MONTEIRO SIQUEIRA,740,06063,07063,No,"46,635","36,905","36,905",4,3,11,"36,00","20,36","56,364",5063,5063,Medicina - Integral - Manaus,2020,22
25551,6651301-4,ESLEYANE MARINHO DE SOUZA,260,02002,03002,No,"37,113","30,952","30,952",5,5,4,"28,00","15,27","43,273",1002,3002,Administração - Vespertino - Manaus,2019,18
37735,6413308-7,TAIANA SILVEIRA DE CASTRO,262,02051,03051,No,"23,797","19,048","19,048",1,1,4,"12,00","16,55","28,545",1051,3051,Licenciatura em Letras - Língua Portuguesa - V...,2019,20
24774,6418070-0,DOUGLAS REIS OLIVEIRA,277,No,No,No,"38,254","32,143","32,143",3,4,5,"24,00","20,36","44,364",3001,3001,Administração - Noturno - Manaus,2019,24
151536,2410735-2,MOISES DA SILVA ARAUJO,86,02025,03025,No,"34,254","32,143","32,143",1,3,4,"16,00","20,36","36,364",1025,2025,Engenharia Mecânica - Integral - Manaus,2022,17
165871,2379920-0,PRISCILA DA SILVA PINHEIRO,189,02075,03075,No,"25,879","16,667","16,667",2,3,3,"16,00","19,09","35,091",1075,3075,Licenciatura em Pedagogia - Matutino - Tefé,2022,23


#### Padronizando as colunas referentes a opção de curso

In [53]:
data['opcao_curso_1'] = data['opcao_curso_1'].astype(str)
data['id_referencia_curso'] = data['id_referencia_curso'].astype(str)

In [54]:
data['opcao_3_curso'] = data['opcao_3_curso'].apply(lambda x: x[1:] if str(x).startswith('0') else x)
data['opcao_curso_2'] = data['opcao_curso_2'].apply(lambda x: x[1:] if str(x).startswith('0') else x)

In [55]:
data.head(8)

,numinscricao,nome,classificacao,opcao_curso_2,opcao_3_curso,habilidades,nota_final,nota_cg,nota_etapa_1,disc1,disc2,disc3,nota_ce,nota_redacao,nota_etapa_2,opcao_curso_1,id_referencia_curso,nome_referencia_curso,ano,idade
0,6354130-0,EDIJANE PINHEIRO MARTINS,313,No,No,No,"31,161","19,048","19,048",3,5,6,"28,00","15,27","43,273",4069,4069,Odontologia - Integral - Manaus,2019,27
1,6318761-2,JOEL DOS SANTOS DE SOUZA,314,No,No,No,"31,154","29,762","29,762",3,1,4,"16,00","16,55","32,545",4069,4069,Odontologia - Integral - Manaus,2019,22
2,6379123-4,CARINA DAS CHAGAS SEIXAS,315,No,No,No,"31,154","29,762","29,762",3,3,2,"16,00","16,55","32,545",4069,4069,Odontologia - Integral - Manaus,2019,23
3,6549853-4,IRLA DA SILVA PEREIRA,316,No,No,No,"31,113","30,952","30,952",4,0,4,"16,00","15,27","31,273",4069,4069,Odontologia - Integral - Manaus,2019,18
4,6393350-0,IVANIR LIMA PEREIRA,317,No,No,No,"31,113","30,952","30,952",2,3,3,"16,00","15,27","31,273",4069,4069,Odontologia - Integral - Manaus,2019,28
5,6612916-8,LAISA DOS SANTOS GALVAO,318,No,No,No,"31,046","25,000","25,000",4,2,3,"18,00","19,09","37,091",4069,4069,Odontologia - Integral - Manaus,2019,16
6,6424859-3,ELCIA NEVES DOS SANTOS,319,No,No,No,"31,046","25,000","25,000",5,3,1,"18,00","19,09","37,091",4069,4069,Odontologia - Integral - Manaus,2019,18
7,6386972-1,LAIZA DE FARIAS MAKLOUF,320,No,No,No,"31,004","26,190","26,190",4,3,2,"18,00","17,82","35,818",4069,4069,Odontologia - Integral - Manaus,2019,17


In [56]:
data.dtypes

numinscricao             object
nome                     object
classificacao             int32
opcao_curso_2            object
opcao_3_curso            object
habilidades              object
nota_final               object
nota_cg                  object
nota_etapa_1             object
disc1                     int64
disc2                     int64
disc3                     int64
nota_ce                  object
nota_redacao             object
nota_etapa_2             object
opcao_curso_1            object
id_referencia_curso      object
nome_referencia_curso    object
ano                       int64
idade                     int64
dtype: object

## Relacionado as informações sobre o curso com cada candidato

Seguindo o padrão observado na extração de dados, id_referencia curso é composto por **XXYYY**. Onde:
- XX: grupo relacionado ao curso
- YY: código do curso

Para conseguir comparar o dataset de candidatos com o de vagas, iremos utilizar somente as informações do grupo (XX) na coluna: 

In [32]:
# data['id_referencia_curso'] = data['id_referencia_curso'].apply(lambda x: x[:1])

In [57]:
data.sample(4)

,numinscricao,nome,classificacao,opcao_curso_2,opcao_3_curso,habilidades,nota_final,nota_cg,nota_etapa_1,disc1,disc2,disc3,nota_ce,nota_redacao,nota_etapa_2,opcao_curso_1,id_referencia_curso,nome_referencia_curso,ano,idade
32410,6382523-6,TAIS DE JESUS RODRIGUES DA COSTA,155,2022,3022,No,"34,362","36,905","36,905",4,1,2,"14,00","17,82","31,818",1022,3022,Engenharia Elétrica - Integral - Manaus,2019,18
93195,8447681-8,PATRICIA KELLY MONTEIRO ALVES,2,7063,No,No,"33,981","32,143","32,143",3,2,4,"18,00","17,82","35,818",6063,6063,Medicina - Integral - Manaus,2020,27
74470,8444224-7,JOAO MATHEUS SOUSA CAMPOS,69,2015,3015,No,"63,754","57,143","57,143",8,6,11,"50,00","20,36","70,364",1015,2015,Direito - Vespertino - Manaus,2020,18
84430,8349981-4,FERNANDA DE ANDRADE BOAVENTURA,5,2041,3041,No,"38,195","28,571","28,571",4,4,7,"30,00","17,82","47,818",1041,3041,Licenciatura em Geografia - Matutino - Tefé,2020,17


Agora, vamos ordenar o dataset por ano, id_referencia_curso e classificação

In [34]:
data.sample(10)

,numinscricao,nome,classificacao,opcao_curso_2,opcao_3_curso,habilidades,nota_final,nota_cg,nota_etapa_1,disc1,disc2,disc3,nota_ce,nota_redacao,nota_etapa_2,opcao_curso_1,id_referencia_curso,nome_referencia_curso,ano,idade
72286,8361304-8,JANDIRA HILARIO HAYDEN,41,2059,3059,No,"29,351","21,429","21,429",6,2,3,"22,00","15,27","37,273",1059,1,Licenciatura em Pedagogia - Noturno - Tabatinga,2020,44
147514,2373800-6,MARIA KAROLINE SANTOS DE SOUZA,261,2002,3002,No,"32,855","22,619","22,619",3,4,5,"24,00","19,09","43,091",1002,2,Administração - Vespertino - Manaus,2022,23
32620,6360717-4,SELNER MONTEIRO SANTOS DA SILVA,133,2023,3023,No,"34,485","33,333","33,333",2,1,4,"14,00","21,64","35,636",1023,3,Engenharia Eletrônica - Noturno - Manaus,2019,23
89855,8319450-9,VANESSA DA SILVA CABRAL,87,6065,7065,No,"44,998","36,905","36,905",3,4,10,"34,00","19,09","53,091",5065,5,Odontologia - Integral - Manaus,2020,18
7200,6677491-8,DIEGO HENRIQUE DE OLIVEIRA LIMA BERNARDO,274,6067,7067,No,"77,046","75,000","75,000",10,9,11,"60,00","19,09","79,091",5067,6,Medicina - Integral - Manaus,2019,26
19446,6418238-0,RICIELY CEZAR SANTOS,201,No,No,No,"26,154","29,762","29,762",2,0,1,"6,00","16,55","22,545",8067,8,Medicina - Integral - Manaus,2019,18
31506,6317774-9,RAFAEL RODRIGUES BATISTA,441,2018,3018,No,"36,766","35,714","35,714",4,4,2,"20,00","17,82","37,818",1018,3,Engenharia da Computação - Integral - Manaus,2019,23
146499,2363472-3,LUCIANA DOS SANTOS FARIAS,18,2093,3093,No,"33,658","30,952","30,952",2,2,4,"16,00","20,36","36,364",1093,1,Tecnologia em Gestão de Turismo - Matutino - Novo,2022,38
103616,1161941-4,DANIELE DE OLIVEIRA AQUINO,24,2067,3067,No,"34,392","20,238","20,238",5,8,3,"32,00","16,55","48,545",1067,1,Licenciatura em Geografia - Matutino - Manaus,2021,17
142628,2336144-1,AGATHA VALENTINA DA SILVA MOREIRA,39,2032,3032,No,"32,236","27,381","27,381",3,2,4,"18,00","19,09","37,091",1032,1,Licenciatura em Ciências Biológicas - Vesperti...,2022,19


In [58]:
year_dataframes = {}
for year in data["ano"].unique():
    df_year = data.loc[data["ano"] == year]
    year_dataframes[year] = df_year

In [124]:
ordered_classifications = {}

for year, df_year in year_dataframes.items():
    year_info = {}
    for course_id in df_year['id_referencia_curso'].unique():
        df_course_year = df_year.loc[df_year['id_referencia_curso'] == course_id]
        df_course_year = df_course_year.sort_values("classificacao")
        year_info[course_id] = df_course_year
    ordered_classifications[year] = year_info

In [125]:
dataframes_ano = []

for ano, id_group in ordered_classifications.items():
    for id in id_group:
      df_ano = pd.DataFrame.from_dict(id_group[id])
      dataframes_year.append(df_year)
df_completo = pd.concat(dataframes_ano, ignore_index=True)

df_completo.to_csv("../data/csv/lista_classificacao_vestibular_completo.csv", index=False)


In [127]:
dados_vaga = pd.read_csv('../data/csv/approved_candidates.csv', sep=',', encoding='utf-8')

In [119]:
df_completo['aprovado'] = df_completo['nome'].isin(dados_vaga['nome'].values).astype(int)

In [130]:
anos_df_completo = df_completo['ano'].unique()
anos_dados_vaga = dados_vaga['ano'].unique()

for ano in anos_df_completo:
    if ano in anos_dados_vaga:
        df_completo_ano = df_completo[df_completo['ano'] == ano]
        dados_vaga_ano = dados_vaga[dados_vaga['ano'] == ano]

        df_completo.loc[df_completo_ano.index, 'aprovado'] = df_completo_ano['nome'].isin(dados_vaga_ano['nome'].values).astype(int)
        df_completo['aprovado'] = df_completo['aprovado'].astype(int)

## Formatando Dataset para treino

In [133]:
columns = ['nota_final', 'nota_ce', 'nota_cg', 'nota_etapa_1', 'nota_etapa_2', 'nota_redacao']

for column in columns:
    df_completo[column] = df_completo[column].str.replace(',', '.').astype(float)


In [136]:
df_completo.sample(2)

,numinscricao,nome,classificacao,opcao_curso_2,opcao_3_curso,habilidades,nota_final,nota_cg,nota_etapa_1,disc1,...,disc3,nota_ce,nota_redacao,nota_etapa_2,opcao_curso_1,id_referencia_curso,nome_referencia_curso,ano,idade,aprovado
178020,2370085-8,FRANCIANA BATISTA COSTA,849,7016,No,No,26.920,15.476,15.476,2,...,5,18.0,20.36,38.364,6016,7016,Enfermagem - Integral - Manaus,2022,39,0
81480,8344652-4,JEAN PIERRE DA CUNHA PATROCINIO,361,2015,3015,No,47.485,33.333,33.333,6,...,10,40.0,21.64,61.636,1015,3015,Direito - Vespertino - Manaus,2020,18,0


In [137]:
df_completo.drop(columns=['nome', 'disc1', 'disc2', 'disc3', 'habilidades'], inplace=True)
df_completo.sample(2)

,numinscricao,classificacao,opcao_curso_2,opcao_3_curso,nota_final,nota_cg,nota_etapa_1,nota_ce,nota_redacao,nota_etapa_2,opcao_curso_1,id_referencia_curso,nome_referencia_curso,ano,idade,aprovado
98993,8485370-0,115,No,No,37.022,30.952,30.952,24.0,19.09,43.091,8063,8063,Medicina - Integral - Manaus,2020,18,0
23078,6341888-6,270,2061,3061,30.582,22.619,22.619,22.0,16.55,38.545,1061,2061,Licenciatura em Pedagogia - Matutino - Parintins,2019,23,0


In [138]:
df_completo.to_csv('../data/dataset.csv', index=False)